In [4]:
# coding: utf-8
import numpy as np
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

In [5]:
# 各種関数を定義しておく
def identity_function(x):
    return x


def step_function(x):
    return np.array(x > 0, dtype=np.int)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))


def sum_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)

In [6]:
#勾配を求める関数
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

In [7]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        np.random.seed(0)
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads

In [8]:
net=TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [9]:
print(net.params['W1'].shape)#一層目の重みは入力サイズ✖️中間層ニューロン数
print(net.params['b1'].shape)#一層目のバイアスは中間層サイズに一致
print(net.params['W2'].shape)#二層目の重みは中間層サイズ✖️出力サイズ
print(net.params['b2'].shape)#二層目のバイアスは出力サイズ

(784, 100)
(100,)
(100, 10)
(10,)


In [13]:
x = np.random.rand(100, 784) # 入力信号：ダミーの入力データ（100 枚分）
t = np.random.rand(100, 10) # ダミーの正解ラベル（100 枚分）

In [16]:
# 各パラメーターに対する勾配を求める
grads = net.gradient(x,t)

In [22]:
print("W1：",grads["W1"])
print("b1：",grads["b1"])
print("W2：",grads["W2"])
print("b2：",grads["b2"])

W1： [[-0.00058953 -0.00039391 -0.00065624 ...  0.00075819  0.00296041
   0.00067478]
 [-0.00053537 -0.00026047 -0.0005211  ...  0.0008718   0.00316288
   0.00078998]
 [-0.00059567 -0.00037249 -0.00059121 ...  0.00067565  0.00293707
   0.00070642]
 ...
 [-0.00064713 -0.00043936 -0.00066541 ...  0.00075356  0.00316515
   0.0007897 ]
 [-0.00062613 -0.00034627 -0.00065367 ...  0.00072738  0.00295474
   0.0006876 ]
 [-0.00065054 -0.00035868 -0.00058051 ...  0.00082982  0.00304716
   0.00075644]]
b1： [-1.19315871e-03 -6.45411449e-04 -1.05634824e-03  2.07875878e-03
 -4.17619340e-04  1.09296066e-03 -2.54246712e-03  1.19449236e-03
  1.61442607e-03  5.45282685e-04 -4.58999752e-04  1.99288616e-03
  1.62778630e-03 -5.30196893e-03 -1.15494752e-03 -1.00293942e-03
 -4.00608671e-03  3.74593684e-03  2.83033859e-03  6.10888029e-03
  3.44084543e-03  3.27121557e-03  1.10824903e-03 -1.51338513e-03
 -1.95372335e-03  1.31322810e-03 -6.26669156e-03  6.78148397e-04
  3.84769771e-03  3.11490211e-03  4.70267664e